In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
print("curren directory:", os.getcwd())

BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/AdversarialAI/'

FIXMATCH_DIR = BASE_DIR + "Fixmatch/"
os.chdir(FIXMATCH_DIR)

print("changed to:", os.getcwd())

curren directory: /content
changed to: /content/drive/MyDrive/Colab Notebooks/AdversarialAI/Fixmatch


# I. Training


## Train configurations

In [3]:
# Train configurations
dataset = 'cifar10'
subset_size = 30000
test_size = 5000
labeled_size = 400
poison_size = 300
seed = 56 # 92 previously

exp_name = 'bird_frog'
poison_output = f"./data/poison/{exp_name}"
out = f'results/{dataset}@{exp_name}'
resume = out + '/checkpoint.pth.tar'

batch_size = 32
max_epoch = 30
eval_steps = subset_size // batch_size + 1
total_steps = eval_steps * max_epoch


# targeted attack option
label_mali = 2 # airplane
label_source = 6 # deer

## Train unpoisoned model

In [ ]:
# Run training command
!python3 train.py --poison-size 0 --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed {seed} --dataset {dataset}

^C


## From scratch

In [ ]:
# Run training command
!python3 train.py --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --poison-size {poison_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed {seed} --dataset {dataset}

## Resume training

In [4]:
# Run training command
!python3 train.py --resume {resume} --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --poison-size {poison_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed 5 --dataset {dataset}

2023-04-23 21:18:28.419016: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 21:18:29.282508: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
['/content/drive/MyDrive/Colab Notebooks/AdversarialAI/Fixmatch', '/content/drive/MyDrive/Colab Notebooks/AdversarialAI/Poisoner', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', './']
04/23/2023 21:18:34 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/23/2023 21:18:34 - INFO - __main__ -   {'gpu_id': 0, 'num_workers': 4, 'dataset': 'cifar10', 'expand_labels': Tr

## Targeted attack training

In [ ]:
# Run training command
!python3 train.py --label-mali {label_mali} --label-source {label_source} --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --poison-size {poison_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed {seed} --dataset {dataset} 

2023-04-23 20:29:12.496613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 20:29:13.367383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
['/content/drive/MyDrive/Colab Notebooks/AdversarialAI/Fixmatch', '/content/drive/MyDrive/Colab Notebooks/AdversarialAI/Poisoner', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', './']
04/23/2023 20:29:15 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/23/2023 20:29:15 - INFO - __main__ -   {'gpu_id': 0, 'num_workers': 4, 'dataset': 'cifar10', 'expand_labels': Tr

# III. Test

In [4]:
import torch
from torch.utils.data import DataLoader, SequentialSampler
import torch.nn.functional as F
from utils import AverageMeter, accuracy
from tqdm import tqdm
import models.wideresnet as models
import time
import logging

import sys
sys.path.insert(1, BASE_DIR + 'Poisoner')
from Poisoner import Poisoner
from get_datasets import get_subset_cifar10
from CustomTransforms import get_fixmatch_transforms

# logger
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

# prepare cuda
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"Using {device}")

# load model
model = models.build_wideresnet(depth=28,
                            widen_factor=2,
                            dropout=0,
                            num_classes=10)
model.to(device)
CHECKPOINT_PATH = "{}/model_best.pth.tar".format(out) 
# CHECKPOINT_PATH = "results/cifar10@poisoned50k.100/model_best.pth.tar"
checkpoint = torch.load(CHECKPOINT_PATH)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print("Loaded model at", CHECKPOINT_PATH)

# load transforms
transform_labeled, transform_unlabeled, transform_test = get_fixmatch_transforms(dataset=dataset)

['/content', '/content/drive/MyDrive/Colab Notebooks/AdversarialAI/Poisoner', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', './']
Using cuda
Loaded model at results/cifar10@bird_frog/model_best.pth.tar
getting transform for cifar10


## Test model on entire test set

In [ ]:
def test_on_testset(test_loader, model, device):
    logger = logging.getLogger(__name__)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            data_time.update(time.time() - end)
            model.eval()

            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.shape[0])
            top1.update(prec1.item(), inputs.shape[0])
            top5.update(prec5.item(), inputs.shape[0])
            batch_time.update(time.time() - end)
            end = time.time()

    logger.info("top-1 acc: {:.2f}".format(top1.avg))
    logger.info("top-5 acc: {:.2f}".format(top5.avg))
    return losses.avg, top1.avg


# load test data
labeled_dataset, unlabeled_dataset, test_dataset = get_subset_cifar10()
test_dataset.transform = transform_test
test_dataset.asPIL = True

test_loader = DataLoader(test_dataset,
                        sampler=SequentialSampler(test_dataset),
                        batch_size=8,
                        num_workers=1)

# test
test_on_testset(test_loader, model, device)

## Test attack success

In [9]:
from PIL import Image
import numpy as np

CHOSEN_POISON_PATH = f'{exp_name}/source_frog.jpg'
# CHOSEN_POISON_PATH = f'{exp_name}/injected/injected_0.001.jpg' # last 0.705, 0.683
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# label_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
# label_names = range(10)

with Image.open(FIXMATCH_DIR + 'data/poison/' + CHOSEN_POISON_PATH) as im:
    transformed_im = transform_test(im).to(device)
    inputs = torch.unsqueeze(transformed_im, dim=0)
    model.eval()
    outputs = model(inputs)
    outputs = torch.softmax(outputs, dim=1)
    print(outputs)
    label_idx = torch.argmax(outputs).cpu().numpy()
    label = label_names[label_idx]
    print("prediction:", label)


# march5night --> Fail 50k.10



tensor([[3.1292e-10, 1.1148e-09, 2.8888e-06, 6.6485e-07, 9.9999e-01, 6.6108e-08,
         1.9455e-06, 1.6962e-06, 2.9600e-09, 1.0438e-08]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
prediction: deer


In [ ]:
labeled_dataset, unlabeled_dataset, test_dataset = get_subset_cifar10()
test_dataset.transform = transform_test
test_dataset.asPIL = True

test_loader = DataLoader(test_dataset,
                        sampler=SequentialSampler(test_dataset),
                        batch_size=8,
                        num_workers=1)


    

In [ ]:
counts = [0]*10
for imgs, labels in test_loader:
    inputs = imgs.to(device)
    outputs = model(inputs)
    outputs = torch.softmax(outputs, dim=1)
    label_indices = torch.argmax(outputs, dim=1).cpu().numpy()
    for idx in label_indices:
        counts[idx] += 1
print(counts)